# Schedule auto shutdown of ACI and compute resources  in the AzureML

Imagine users deploying models through AutoML and you want to schedule a shutdown of them every night, just in case your power users forgot to close them on their own.
This pipeline will shutdown all ACI in the AzureML's resource group and all compute instances running in the AzureML workspace. It will also scale all clusters down to 0 nodes.

Potential variations of this script could include:
- Filtering the ACIs to close based on resource tags (e.g. if resource has tag environment:prod avoid shutting down).
- Create a single script step that invokes both scripts to avoid spinning up 2 nodes on a multi node cluster.


In [ ]:
# Variables used in script
compute_cluster_name='shutdown-cluster'
pipeline_name='shutdown-resources-pipeline'

Make sure that you provide the compute clusters managed identity the following permissions:

- Microsoft.ContainerInstance/containerGroups/read : To read ACI resources
- Microsoft.ContainerInstance/containerGroups/stop/action : To stop ACI
- Microsoft.MachineLearningServices/workspaces/read : To get compute instances
- Microsoft.MachineLearningServices/workspaces/computes/read : To get compute instances
- Microsoft.MachineLearningServices/workspaces/computes/stop/action : To stop compute instances
- Microsoft.MachineLearningServices/workspaces/computes/write : To be able to resize clusters

If you want to create a custom role:

```
{
    "properties": {
        "roleName": "shutdownazuremlresources",
        "description": "",
        "assignableScopes": [
            "/subscriptions/<subscription id>/resourceGroups/<resource group name>"
        ],
        "permissions": [
            {
                "actions": [
                    "Microsoft.ContainerInstance/containerGroups/stop/action",
                    "Microsoft.ContainerInstance/containerGroups/read",
                    "Microsoft.MachineLearningServices/workspaces/read",
                    "Microsoft.MachineLearningServices/workspaces/computes/read",
                    "Microsoft.MachineLearningServices/workspaces/computes/stop/action",
                    "Microsoft.MachineLearningServices/workspaces/computes/write"
                ],
                "notActions": [],
                "dataActions": [],
                "notDataActions": []
            }
        ]
    }
}
```

The name of the system assigned managed identity should be `<workspace name>/computes/<cluster name>`

Otherwise you will see the following error in 70_driver_log.txt:
```
Message: The client <guid> with object id <guid> does not have authorization to perform action 
Microsoft.ContainerInstance/containerGroups/read over scope 
/subscriptions/<subscription id>/resourceGroups/<resource group>/providers/Microsoft.ContainerInstance or the scope is invalid. 
If access was recently granted, please refresh your credentials.
```

Navigate to //portal.azure.com/#blade/Microsoft_AAD_IAM/ManagedAppMenuBlade/Overview/appId/`<guid>`/objectId/`<guid>` to see the system assigned managed identity of that compute.

In [ ]:
from azureml.core import Workspace
from azureml.core.compute import ComputeTarget

# Connect to workspace and get resource references
ws = Workspace.from_config()
compute_cluster = ComputeTarget(workspace=ws, name=compute_cluster_name)

In [ ]:
from azureml.core import Environment
from azureml.core.runconfig import RunConfiguration
# Create an environment with the pip requirements
execution_environment = Environment.from_pip_requirements(f"{pipeline_name}-environment", 'requirements.txt')
# Register and refresh environment
execution_environment.register(workspace=ws)
building_process = execution_environment.build(workspace=ws)
building_process.wait_for_completion(show_output=True)
# Create a run config that we will use in our steps
execution_run_config = RunConfiguration()
execution_run_config.environment = execution_environment

In [ ]:
from azureml.pipeline.steps import PythonScriptStep
# Step to close ACI
step_0 = PythonScriptStep(
    'close_azure_container_instances.py',
    source_directory='.',
    name='Close ACI resources',
    compute_target=compute_cluster,
    runconfig=execution_run_config,
    allow_reuse= False
)

# Step to close compute instances
step_1 = PythonScriptStep(
    'close_azureml_compute_resources.py',
    source_directory='.',
    name='Close AzureML computes',
    compute_target=compute_cluster,
    runconfig=execution_run_config,
    allow_reuse= False
)

In [ ]:
from azureml.pipeline.core import Pipeline

pipeline = Pipeline(workspace=ws, steps=[step_0, step_1])

published_pipeline = pipeline.publish(
    pipeline_name, 
    description="Shutdown AzureML resources")

In [ ]:
from azureml.pipeline.core import PublishedPipeline
from azureml.pipeline.core.schedule import Schedule

# Clean up previous deployments
published_pipelines = PublishedPipeline.list(workspace=ws, active_only=True)
schedules = Schedule.list(ws, active_only=True)

for p in published_pipelines:
    if (p.name == pipeline_name and p.id != published_pipeline.id):
        print(f"Disabling pipeline with id {p.id} and name {p.name}")
        for s in schedules:
            if (s.pipeline_id == p.id):
                print(f"\tDisabling schedule {s.id}")
                s.disable(wait_for_provisioning=True)
        # Once scheduled pipelines are cleared, disable pipeline
        p.disable()

In [ ]:
from azureml.pipeline.core.schedule import ScheduleRecurrence, Schedule, TimeZone

recurrence = ScheduleRecurrence(frequency="Day", 
                                interval=1, 
                                hours=[22], 
                                minutes=[30],
                                time_zone=TimeZone.EEuropeStandardTime,
                                # start_time = datetime.now()  # This will prevent starting the pipeline once it's scheduled
                                ) # Runs every other day at 10:30pm

schedule = Schedule.create(workspace=ws, name=f"{pipeline_name}-schedule",
                           pipeline_id=published_pipeline.id, 
                           experiment_name=f"{pipeline_name}-schedule-run",
                           recurrence=recurrence,
                           wait_for_provisioning=True,
                           description="Shutdown AzureML related resources")

# You may want to make sure that the schedule is provisioned properly
# before making any further changes to the schedule

print("Created schedule with id: {}".format(schedule.id))

In [ ]:
from azureml.pipeline.core.schedule import Schedule
# Use active_only=False to get all schedules including disabled schedules
schedules = Schedule.list(ws, active_only=True) 
print("Your workspace has the following schedules set up:")
for schedule in schedules:
    print("{} (Published pipeline: {}".format(schedule.id, schedule.pipeline_id))
    # schedule.disable(wait_for_provisioning=True)